### Speed up YOLOv4 inference to twice as fast on Amazon SageMaker
From the blog post found here:   
https://aws.amazon.com/blogs/machine-learning/speed-up-yolov4-inference-to-twice-as-fast-on-amazon-sagemaker/

In [1]:
# make sure we have the latest version of sagemaker installed
!pip install sagemaker --upgrade --quiet

In [2]:
import numpy as np
import time
import json
import requests
import boto3
import os
import sagemaker

In [3]:
role = sagemaker.get_execution_role()
sess = sagemaker.session.Session()
region = sess.boto_region_name
bucket = sess.default_bucket()

import torch
print(torch.__version__)

1.6.0


In [4]:
import sys
print(sys.version)

3.6.13 |Anaconda, Inc.| (default, Feb 23 2021, 21:15:04) 
[GCC 7.3.0]


In [5]:
model_archive = 'yolov4.tar.gz'
!wget https://aws-ml-blog-artifacts.s3.us-east-2.amazonaws.com/yolov4.tar.gz

--2021-09-21 19:31:12--  https://aws-ml-blog-artifacts.s3.us-east-2.amazonaws.com/yolov4.tar.gz
Resolving aws-ml-blog-artifacts.s3.us-east-2.amazonaws.com (aws-ml-blog-artifacts.s3.us-east-2.amazonaws.com)... 52.219.106.114
Connecting to aws-ml-blog-artifacts.s3.us-east-2.amazonaws.com (aws-ml-blog-artifacts.s3.us-east-2.amazonaws.com)|52.219.106.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 239656714 (229M) [application/x-gzip]
Saving to: ‘yolov4.tar.gz’

yolov4.tar.gz       100%[===================>] 228.55M  41.1MB/s    in 5.4s    

2021-09-21 19:31:20 (42.7 MB/s) - ‘yolov4.tar.gz’ saved [239656714/239656714]



In [7]:
from sagemaker.utils import name_from_base

compilation_job_name = name_from_base('torchvision-yolov4-neo-1')
prefix = compilation_job_name+'/model'
model_path = sess.upload_data(path=model_archive, key_prefix=prefix)
compiled_model_path = 's3://{}/{}/output'.format(bucket, compilation_job_name)

In [ ]:
# deploy endpoint

framework_version = '1.6'
py_version = 'py3'
instance_type = 'ml.c5.9xlarge'
from sagemaker.pytorch.model import PyTorchModel
from sagemaker.predictor import Predictor

sm_model = PyTorchModel(
    model_data=model_path,
    framework_version=framework_version,
    role=role,
    sagemaker_session=sess,
    entry_point='code/inference.py',
    py_version=py_version,
    env={"COMPILEDMODEL": 'False', 'MMS_MAX_RESPONSE_SIZE': '100000000', 'MMS_DEFAULT_RESPONSE_TIMEOUT': '500'}
)

uncompiled_predictor = sm_model.deploy(
    initial_instance_count=1, 
    instance_type=instance_type
)

---

In [ ]:
input_layer_name = 'input0'
input_shape = [1,3,416,416]
data_shape = json.dumps({input_layer_name: input_shape})
target_device = 'ml_c5'
framework = 'PYTORCH'
compiled_env = {"MMS_DEFAULT_WORKERS_PER_MODEL":'1', "TVM_NUM_THREADS": '36', "COMPILEDMODEL": 'True', 'MMS_MAX_RESPONSE_SIZE': '100000000', 'MMS_DEFAULT_RESPONSE_TIMEOUT': '500'}

sm_model_compiled = PyTorchModel(
    model_data=model_path,
    framework_version = framework_version,
    role=role,
    sagemaker_session=sess,
    entry_point='code/inference.py',
    py_version=py_version,
    env=compiled_env
)

compiled_model = sm_model_compiled.compile(
    target_instance_family=target_device, 
    input_shape=data_shape,
    job_name=compilation_job_name,
    role=role,
    framework=framework.lower(),
    framework_version=framework_version,
    output_path=compiled_model_path
)

optimized_predictor = compiled_model.deploy(
    initial_instance_count = 1,
    instance_type = instance_type
)

In [ ]:
# make predictions

iters = 1000
warmup = 100
client = boto3.client('sagemaker-runtime', region_name=region)

content_type = 'application/x-image'

sample_img_url = "https://github.com/ultralytics/yolov5/raw/master/data/images/zidane.jpg"
body = requests.get(sample_img_url).content
   
compiled_perf = []
uncompiled_perf = []
  
for i in range(iters):
    t0 = time.time()
    response = client.invoke_endpoint(EndpointName=optimized_predictor.endpoint_name, Body=body, ContentType=content_type)
    t1 = time.time()
    #convert to millis
    compiled_elapsed = (t1-t0)*1000

    t0 = time.time()
    response = client.invoke_endpoint(EndpointName=uncompiled_predictor.endpoint_name, Body=body, ContentType=content_type)
    t1 = time.time()
    #convert to millis
    uncompiled_elapsed = (t1-t0)*1000
    
    if warmup == 0:
        compiled_perf.append(compiled_elapsed)
        uncompiled_perf.append(uncompiled_elapsed)
    else:
        print(f'warmup ({i}, {iters}) : c - {compiled_elapsed} ms . uc - {uncompiled_elapsed} ms')
        warmup -= 1
        
        

In [ ]:
import matplotlib.pyplot as plt

plt.plot(compiled_perf, 'red')
plt.plot(uncompiled_perf, 'blue')
plt.xlabel('Iteration')
plt.ylabel('Milliseconds')
plt.legend(["Compiled", "Uncompiled"], loc=0)
plt.show()

